In [1]:
import openai

In [2]:
f = open('/Users/ericmelz/Documents/code/openai-quickstart-python-emelz/.env', 'r')

KEY='OPENAI_API_KEY'
l=f.readline()
api_key = 'undefined!'
while l:
    if l.startswith(KEY):
        print('FOUND')
        api_key = l[len(KEY)+1:-1]
        break
    l = f.readline()

f.close()

openai.api_key=api_key

FOUND


In [3]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Josh decides to try flipping a house.  He buys a house for $80,000 and then puts in $50,000 in repairs.  This increased the value of the house by 150%.  How much profit did he make?"},
    ]
)

In [4]:
response

<OpenAIObject chat.completion id=chatcmpl-8DMzuOHtMtyVc50GRfdKP5nWr8KTb at 0x10423a900> JSON: {
  "id": "chatcmpl-8DMzuOHtMtyVc50GRfdKP5nWr8KTb",
  "object": "chat.completion",
  "created": 1698197358,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The value of the house increased by 150%, so this means there was an increase of 150/100 = $<<150/100*80000=120000>>120,000\nSo the total value of the house was 80,000+50,000 = $<<80000+50000=130000>>130,000\nThis means the value of the house increased to 130,000+120,000 = $<<130000+120000=250000>>250,000\nHis profit was 250,000-130,000 = $<<250000-130000=120000>>120,000. Answer: \\boxed{120000}."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 56,
    "completion_tokens": 139,
    "total_tokens": 195
  }
}

In [5]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Josh decides to try flipping a house.  He buys a house for $80,000 and then puts in $50,000 in repairs.  This increased the value of the house by 150%.  How much profit did he make?"},
    ]
)

In [6]:
response

<OpenAIObject chat.completion id=chatcmpl-8DN5fj2F09odoZiSv1qnKS1PqM9Nq at 0x10423a220> JSON: {
  "id": "chatcmpl-8DN5fj2F09odoZiSv1qnKS1PqM9Nq",
  "object": "chat.completion",
  "created": 1698197715,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The increase in value was 80000*1.5=$<<80000*1.5=120000>>120,000\nSo the house is now worth 120000+80000=$<<120000+80000=200000>>200,000\nSo he made a profit of 200000-80000-50000=$<<200000-80000-50000=70000>>70,000. Answer: \\boxed{70000}."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 56,
    "completion_tokens": 96,
    "total_tokens": 152
  }
}